In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Chargement des 4 dataframes précedement préparés

In [2]:
df4 = pd.read_csv('df4.csv', delimiter = ',', encoding="utf8",index_col = 0)
df4.name = 'dataframe_4encoded'

df3 = pd.read_csv('df3.csv', delimiter = ',', encoding="utf8",index_col = 0)
df3.name = 'dataframe_3encoded'

df_log3 = pd.read_csv('df_log3.csv', delimiter = ',', encoding="utf8",index_col = 0)
df_log3.name='dataframe_3encoded_log'

df_log4 = pd.read_csv('df_log4.csv', delimiter = ',', encoding="utf8",index_col = 0)
df_log4.name='dataframe_4encoded_log'

pd.set_option("max_colwidth", None)

### Creation d'une fonction permettant a partir du model choisi et de ses parametres d'effectuer un grid search sur tous nos datasets avec les 2 variables cibles qui sont 'TotalGHGEmissions' et 'SiteEnergyUse(kBtu)'

In [3]:
datasets = [df3,df4,df_log3,df_log4]
def bestmod(model,params):
    import timeit
    import datetime
    debut = datetime.datetime.now()
    
    d_namex = []
    d_shapex = []
    targx = []
    r2_testx = []
    r2_trainx = []
    best_px = []
    best_sx = []
    model_n = []
    
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_absolute_error,mean_squared_error
    
    print("RESULTATS GRID SEARCH : ")
    model_name = type(model).__name__
    print(model_name,"\n")
        
    for dataset in datasets:
        X = dataset.drop(['TotalGHGEmissions','ENERGYSTARScore','SiteEnergyUse(kBtu)'],axis = 1)
        y = dataset['TotalGHGEmissions']
                 
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)
        std_scale = preprocessing.StandardScaler().fit(X_train)
        X_train_std = std_scale.transform(X_train)
        X_test_std = std_scale.transform(X_test)
                       
        grid = GridSearchCV(model,param_grid=params,n_jobs=-1)
        grid.fit(X_train_std,y_train)
        y_pred = grid.predict(X_test_std)
        r2_test = round((r2_score(y_test,y_pred)),3)
        y_pred2 = grid.predict(X_train_std)
        r2_train = round((r2_score(y_train,y_pred2)),3)
            
        d_namex.append(dataset.name)
        d_shapex.append(dataset.shape)
        targx.append('TotalGHGEmissions')
        r2_testx.append(r2_test)
        r2_trainx.append(r2_train)
        best_px.append(grid.best_params_)
        best_sx.append(round(grid.best_score_,3))
        model_n.append(model_name)
        
        ####################+ENERGYSTARScore en entree################           
        temp2 = dataset.dropna(subset=['ENERGYSTARScore'],how='any')
        X2 = temp2.drop(['TotalGHGEmissions','SiteEnergyUse(kBtu)'],axis = 1)
        y2 = temp2['TotalGHGEmissions']
            
        X_train2,X_test2,y_train2,y_test2 = train_test_split(X2,y2,test_size = 0.3,random_state = 42)
        std_scale2 = preprocessing.StandardScaler().fit(X_train2)
        X_train_std2 = std_scale2.transform(X_train2)
        X_test_std2 = std_scale2.transform(X_test2)

        grid2 = GridSearchCV(model,param_grid=params,n_jobs=-1)
        grid2.fit(X_train_std2,y_train2)
        y_pred21 = grid2.predict(X_test_std2)
        r2_test2 = round((r2_score(y_test2,y_pred21)),3)
        y_pred22 = grid2.predict(X_train_std2)
        r2_train2 = round((r2_score(y_train2,y_pred22)),3)
            
        d_namex.append(dataset.name)
        d_shapex.append(temp2.shape)
        targx.append('TotalGHGEm(+in ES)')
        r2_testx.append(r2_test2)
        r2_trainx.append(r2_train2)
        best_px.append(grid2.best_params_)
        best_sx.append(round(grid2.best_score_,3))
        model_n.append(model_name)
        
        #############################################################
        temp3 = dataset.copy()
        X3 = temp3.drop(['TotalGHGEmissions','ENERGYSTARScore','SiteEnergyUse(kBtu)'],axis = 1)
        y3 = temp3['SiteEnergyUse(kBtu)']
            
        X_train3,X_test3,y_train3,y_test3 = train_test_split(X3,y3,test_size = 0.3,random_state = 42)
        std_scale3 = preprocessing.StandardScaler().fit(X_train3)
        X_train_std3 = std_scale3.transform(X_train3)
        X_test_std3 = std_scale3.transform(X_test3)

        grid3 = GridSearchCV(model,param_grid=params,n_jobs=-1)
        grid3.fit(X_train_std3,y_train3)
        y_pred31 = grid3.predict(X_test_std3)
        r2_test3 = round((r2_score(y_test3,y_pred31)),3)
        y_pred32 = grid3.predict(X_train_std3)
        r2_train3 = round((r2_score(y_train3,y_pred32)),3)
            
        d_namex.append(dataset.name)
        d_shapex.append(temp3.shape)
        targx.append('SiteEnergyUse(kBtu)')
        r2_testx.append(r2_test3)
        r2_trainx.append(r2_train3)
        best_px.append(grid3.best_params_)
        best_sx.append(round(grid3.best_score_,3))
        model_n.append(model_name)
        
        #############################################################
    df_res = pd.DataFrame()
    df_res['dataset_name'] = d_namex
    df_res['dataset_shape'] = d_shapex
    df_res['target'] = targx
    df_res['best_sc'] = best_sx
    df_res['r2_test'] = r2_testx
    df_res['r2_train'] = r2_trainx
    df_res['best_pa'] = best_px
    df_res['model_name'] = model_n
            
    fin = datetime.datetime.now()
    print("temps:_________________",fin-debut)
    
    return df_res

## Regression lineaire

In [4]:
from sklearn.linear_model import LinearRegression
model1 = LinearRegression()
params1 ={"fit_intercept":[True]}

In [5]:
results1 = bestmod(model1,params1)

RESULTATS GRID SEARCH : 
LinearRegression 

temps:_________________ 0:00:05.303484


In [6]:
results1

,dataset_name,dataset_shape,target,best_sc,r2_test,r2_train,best_pa,model_name
0,dataframe_3encoded,"(1595, 54)",TotalGHGEmissions,-3.154019e+23,-1.070067e+25,0.480,{'fit_intercept': True},LinearRegression
1,dataframe_3encoded,"(1059, 54)",TotalGHGEm(+in ES),-1.388625e+24,6.200000e-02,0.638,{'fit_intercept': True},LinearRegression
2,dataframe_3encoded,"(1595, 54)",SiteEnergyUse(kBtu),-1.108494e+23,-2.322071e+25,0.552,{'fit_intercept': True},LinearRegression
3,dataframe_4encoded,"(1582, 108)",TotalGHGEmissions,-2.565869e+27,-4.628387e+24,0.452,{'fit_intercept': True},LinearRegression
4,dataframe_4encoded,"(1054, 108)",TotalGHGEm(+in ES),-4.100871e+26,-8.557255e+25,0.771,{'fit_intercept': True},LinearRegression
5,dataframe_4encoded,"(1582, 108)",SiteEnergyUse(kBtu),-1.186818e+27,-5.476758e+23,0.581,{'fit_intercept': True},LinearRegression
6,dataframe_3encoded_log,"(1059, 54)",TotalGHGEmissions,-2.950671e+25,5.680000e-01,0.610,{'fit_intercept': True},LinearRegression
7,dataframe_3encoded_log,"(1059, 54)",TotalGHGEm(+in ES),-1.007857e+24,6.280000e-01,0.663,{'fit_intercept': True},LinearRegression
8,dataframe_3encoded_log,"(1059, 54)",SiteEnergyUse(kBtu),-1.417615e+25,7.710000e-01,0.694,{'fit_intercept': True},LinearRegression
9,dataframe_4encoded_log,"(1054, 108)",TotalGHGEmissions,-6.680353e+26,-2.322651e+26,0.590,{'fit_intercept': True},LinearRegression


##  SVM

In [7]:
from sklearn.svm import SVR
model2 = SVR()
params2 = {'C':np.arange(10,85, 5),
             'kernel':['linear','rbf','poly'],
              'gamma':['scale','auto'],
              'degree':[1,2,3],
             'coef0':np.arange(0.1,0.9, 0.1)}

In [8]:
results2 = bestmod(model2,params2)

RESULTATS GRID SEARCH : 
SVR 

temps:_________________ 2:21:13.675436


In [9]:
results2

,dataset_name,dataset_shape,target,best_sc,r2_test,r2_train,best_pa,model_name
0,dataframe_3encoded,"(1595, 54)",TotalGHGEmissions,0.465,0.711,0.538,"{'C': 80, 'coef0': 0.8, 'degree': 3, 'gamma': 'auto', 'kernel': 'poly'}",SVR
1,dataframe_3encoded,"(1059, 54)",TotalGHGEm(+in ES),0.595,0.494,0.983,"{'C': 80, 'coef0': 0.8, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}",SVR
2,dataframe_3encoded,"(1595, 54)",SiteEnergyUse(kBtu),-0.080,-0.067,-0.046,"{'C': 80, 'coef0': 0.1, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}",SVR
3,dataframe_4encoded,"(1582, 108)",TotalGHGEmissions,0.359,0.677,0.313,"{'C': 80, 'coef0': 0.8, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}",SVR
4,dataframe_4encoded,"(1054, 108)",TotalGHGEm(+in ES),0.424,0.703,0.552,"{'C': 15, 'coef0': 0.5, 'degree': 3, 'gamma': 'auto', 'kernel': 'poly'}",SVR
5,dataframe_4encoded,"(1582, 108)",SiteEnergyUse(kBtu),-0.073,-0.061,-0.045,"{'C': 80, 'coef0': 0.1, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}",SVR
6,dataframe_3encoded_log,"(1059, 54)",TotalGHGEmissions,0.526,0.536,0.585,"{'C': 10, 'coef0': 0.5, 'degree': 1, 'gamma': 'auto', 'kernel': 'poly'}",SVR
7,dataframe_3encoded_log,"(1059, 54)",TotalGHGEm(+in ES),0.584,0.607,0.642,"{'C': 10, 'coef0': 0.2, 'degree': 1, 'gamma': 'auto', 'kernel': 'poly'}",SVR
8,dataframe_3encoded_log,"(1059, 54)",SiteEnergyUse(kBtu),0.681,0.771,0.675,"{'C': 10, 'coef0': 0.5, 'degree': 1, 'gamma': 'scale', 'kernel': 'poly'}",SVR
9,dataframe_4encoded_log,"(1054, 108)",TotalGHGEmissions,0.531,0.571,0.595,"{'C': 10, 'coef0': 0.4, 'degree': 1, 'gamma': 'auto', 'kernel': 'poly'}",SVR


## DecisionTreeRegressor

In [10]:
from sklearn.tree import DecisionTreeRegressor
model3 = DecisionTreeRegressor()
params3 = {
    'criterion':['absolute_error'],
    'splitter':['best'],
    'max_leaf_nodes': np.arange(1,15, 1),
    'ccp_alpha': np.arange(0.1,0.6, 0.1),
    'max_depth' : np.arange(2,22, 2)
    }

In [ ]:
results3 = bestmod(model3,params3)

In [12]:
results3

,dataset_name,dataset_shape,target,best_sc,r2_test,r2_train,best_pa,model_name
0,dataframe_3encoded,"(1595, 54)",TotalGHGEmissions,0.346,0.323,0.476,"{'ccp_alpha': 0.4, 'criterion': 'absolute_error', 'max_depth': 4, 'max_leaf_nodes': 12, 'splitter': 'best'}",DecisionTreeRegressor
1,dataframe_3encoded,"(1059, 54)",TotalGHGEm(+in ES),0.232,0.392,0.530,"{'ccp_alpha': 0.30000000000000004, 'criterion': 'absolute_error', 'max_depth': 16, 'max_leaf_nodes': 13, 'splitter': 'best'}",DecisionTreeRegressor
2,dataframe_3encoded,"(1595, 54)",SiteEnergyUse(kBtu),0.483,0.549,0.466,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 4, 'max_leaf_nodes': 4, 'splitter': 'best'}",DecisionTreeRegressor
3,dataframe_4encoded,"(1582, 108)",TotalGHGEmissions,0.219,0.479,0.169,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 6, 'splitter': 'best'}",DecisionTreeRegressor
4,dataframe_4encoded,"(1054, 108)",TotalGHGEm(+in ES),0.342,-0.230,0.412,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 2, 'splitter': 'best'}",DecisionTreeRegressor
5,dataframe_4encoded,"(1582, 108)",SiteEnergyUse(kBtu),0.403,0.314,0.312,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 4, 'splitter': 'best'}",DecisionTreeRegressor
6,dataframe_3encoded_log,"(1059, 54)",TotalGHGEmissions,0.252,0.357,0.272,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 2, 'splitter': 'best'}",DecisionTreeRegressor
7,dataframe_3encoded_log,"(1059, 54)",TotalGHGEm(+in ES),0.252,0.357,0.272,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 2, 'splitter': 'best'}",DecisionTreeRegressor
8,dataframe_3encoded_log,"(1059, 54)",SiteEnergyUse(kBtu),0.398,0.493,0.377,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 3, 'splitter': 'best'}",DecisionTreeRegressor
9,dataframe_4encoded_log,"(1054, 108)",TotalGHGEmissions,0.260,0.301,0.283,"{'ccp_alpha': 0.1, 'criterion': 'absolute_error', 'max_depth': 2, 'max_leaf_nodes': 2, 'splitter': 'best'}",DecisionTreeRegressor


##  adaboost

In [18]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

model4 = AdaBoostRegressor()
params4 = {'base_estimator': [RandomForestRegressor(), LinearRegression(), SVR()],
             'learning_rate': np.arange(0.05,0.80, 0.05),
             'n_estimators': np.arange(1,13, 1)}
#(*np.arange(0.01, 1, 0.1), *np.arange(1,200,10))

In [9]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

model4 = AdaBoostRegressor()
params4 = {'base_estimator': [RandomForestRegressor(), LinearRegression(), SVR()],
             'learning_rate': [0.01,0.05,0.1,0.3,0.5,0.7, 1],
             'n_estimators': np.arange(10,300,10)}

In [ ]:
results4 = bestmod(model4,params4)

In [11]:
results4

,dataset_name,dataset_shape,target,best_sc,r2_test,r2_train,best_pa,model_name
0,dataframe_3encoded,"(1595, 54)",TotalGHGEmissions,0.274,0.676,0.899,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 10}",AdaBoostRegressor
1,dataframe_3encoded,"(1059, 54)",TotalGHGEm(+in ES),0.410,0.753,0.998,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.7, 'n_estimators': 10}",AdaBoostRegressor
2,dataframe_3encoded,"(1595, 54)",SiteEnergyUse(kBtu),0.374,0.620,0.915,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 10}",AdaBoostRegressor
3,dataframe_4encoded,"(1582, 108)",TotalGHGEmissions,0.137,0.669,0.543,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 10}",AdaBoostRegressor
4,dataframe_4encoded,"(1054, 108)",TotalGHGEm(+in ES),-0.012,-0.029,-0.011,"{'base_estimator': SVR(), 'learning_rate': 0.3, 'n_estimators': 130}",AdaBoostRegressor
5,dataframe_4encoded,"(1582, 108)",SiteEnergyUse(kBtu),0.467,0.586,0.690,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 10}",AdaBoostRegressor
6,dataframe_3encoded_log,"(1059, 54)",TotalGHGEmissions,0.524,0.577,0.936,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 90}",AdaBoostRegressor
7,dataframe_3encoded_log,"(1059, 54)",TotalGHGEm(+in ES),0.602,0.647,0.944,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 80}",AdaBoostRegressor
8,dataframe_3encoded_log,"(1059, 54)",SiteEnergyUse(kBtu),0.626,0.771,0.960,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.01, 'n_estimators': 60}",AdaBoostRegressor
9,dataframe_4encoded_log,"(1054, 108)",TotalGHGEmissions,0.558,0.557,0.949,"{'base_estimator': RandomForestRegressor(), 'learning_rate': 0.05, 'n_estimators': 50}",AdaBoostRegressor


In [14]:
results_all34 = pd.concat([results1,results2,results3,results4], axis=0, ignore_index=True)
results_all34

,dataset_name,dataset_shape,target,best_sc,r2_test,r2_train,best_pa,model_name
0,dataframe_3encoded,"(1595, 54)",TotalGHGEmissions,-3.154019e+23,-1.070067e+25,0.480,{'fit_intercept': True},LinearRegression
1,dataframe_3encoded,"(1059, 54)",TotalGHGEm(+in ES),-1.388625e+24,6.200000e-02,0.638,{'fit_intercept': True},LinearRegression
2,dataframe_3encoded,"(1595, 54)",SiteEnergyUse(kBtu),-1.108494e+23,-2.322071e+25,0.552,{'fit_intercept': True},LinearRegression
3,dataframe_4encoded,"(1582, 108)",TotalGHGEmissions,-2.565869e+27,-4.628387e+24,0.452,{'fit_intercept': True},LinearRegression
4,dataframe_4encoded,"(1054, 108)",TotalGHGEm(+in ES),-4.100871e+26,-8.557255e+25,0.771,{'fit_intercept': True},LinearRegression
5,dataframe_4encoded,"(1582, 108)",SiteEnergyUse(kBtu),-1.186818e+27,-5.476758e+23,0.581,{'fit_intercept': True},LinearRegression
6,dataframe_3encoded_log,"(1059, 54)",TotalGHGEmissions,-2.950671e+25,5.680000e-01,0.610,{'fit_intercept': True},LinearRegression
7,dataframe_3encoded_log,"(1059, 54)",TotalGHGEm(+in ES),-1.007857e+24,6.280000e-01,0.663,{'fit_intercept': True},LinearRegression
8,dataframe_3encoded_log,"(1059, 54)",SiteEnergyUse(kBtu),-1.417615e+25,7.710000e-01,0.694,{'fit_intercept': True},LinearRegression
9,dataframe_4encoded_log,"(1054, 108)",TotalGHGEmissions,-6.680353e+26,-2.322651e+26,0.590,{'fit_intercept': True},LinearRegression


### Sauvegarde du dataframe 'results_all34' contenant tous les meilleurs resultats de toutes les regressions effectuées avec les 4 dataframes avec gridsearchCV.

In [13]:
results_all34.to_csv('results_all34.csv')